In [2]:
import cv2
import numpy as np

In [4]:
image = cv2.imread(r'E:/Books/2021-11-23 10_48_26-USING COCO DATASET - Jupyter Notebook.png')
cv2.imshow('original_image',image)

cv2.waitKey(0)
cv2.destroyWindow('original_image')

w,h = image.shape[:2]

In [5]:
blob = cv2.dnn.blobFromImage(image,1./255,(416,416),swapRB = True,crop = False)

print(image.shape)
print(blob.shape)
blob_to_show = blob[0,:,:,:].transpose(1,2,0)

print(blob_to_show.shape)

cv2.namedWindow('blob to show',cv2.WINDOW_NORMAL)
cv2.imshow('blob to show',cv2.cvtColor(blob_to_show,cv2.COLOR_BGR2RGB))

cv2.waitKey(0)
cv2.destroyAllWindows()

(1017, 824, 3)
(1, 3, 416, 416)
(416, 416, 3)


In [6]:
with open(r'coco.names') as f:
    labels = [line.strip() for line in f]

print(labels)

FileNotFoundError: [Errno 2] No such file or directory: 'coco.names'

In [7]:
network = cv2.dnn.readNetFromDarknet('yolov3.cfg','yolov3.weights')

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\dnn\src\darknet\darknet_importer.cpp:207: error: (-212:Parsing error) Failed to parse NetParameter file: yolov3.cfg in function 'cv::dnn::dnn4_v20200609::readNetFromDarknet'


In [8]:
layers_all = network.getLayerNames()

output_layers = [layers_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]

print(output_layers)

threshould = 0.7
probability = 0.6

color = np.random.randint(0,255,size = (len(labels),3), dtype= 'int')

NameError: name 'network' is not defined

In [ ]:
print(color[1])

In [ ]:
network.setInput(blob)
output = network.forward(output_layers)
print(output)

In [ ]:
bounding_boxes = []
probabilities = []
class_numbers = []

for i in output:
    for detected_object in i:
        scores = detected_object[:5]
        indicies_of_maxscore = np.argmax(scores)
        class_scores = scores[indicies_of_maxscore]
        
        
        if class_scores>probability:
            box_current = detected_object[0:4] * ([w,h,w,h])
            x_center,y_center,height,width = box_current
            x_min = (x_center - (width/2))
            y_min = (y_center - (height/2))
            
            bounding_boxes.append([x_min,y_min,int(width),int(height)])
            probabilities.append(float(class_scores))
            class_numbers.append(indicies_of_maxscore)

In [ ]:
nmsboxes = cv2.dnn.NMSBoxes(bounding_boxes,probabilities,probability,threshould)